In [1]:
using Laplacians

objc[5931]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[5931]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[5931]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[5931]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [19]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
a = readFromFile("../graphs/se_1/graph.mtx");

In [5]:
tree = readFromFile("../graphs/se_1/tree.mtx");

In [6]:
x = readFromFile("../graphs/se_1/x.vec");

In [7]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [8]:
norm(lap(a) * x - b) / norm(b)

0.0

In [9]:
la = lap(a);

In [14]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [15]:
@time F(b);

  0.059051 seconds (28.49 k allocations: 1.156 MB)


In [16]:
@time myx64 = pcg(la, b, F, tol=1e-6, maxits=10000, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

PCG stopped after: 1665 iterations with relative error 7.668194673886251e-7.
  1.651447 seconds (22.49 M allocations: 829.928 MB, 5.70% gc time)


7.66819467033642e-7

In [21]:
dbg = []

for hp in [53,64,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
#     @time myxHP = pcgVMatNorm(laHP, bHP, FHP, xHP, tol=BigFloat(1e-2), maxits=1000, verbose=true);
    @time myxHP = pcgV(laHP, bHP, FHP, tol=BigFloat(1e-6), maxits=10000, verbose=true);
    println("Error is: ", Float64(norm(laHP * myxHP - bHP) / norm(bHP)))
    push!(dbg, "")
    push!(dbg, "")
    
end

Working on iteration 100
Working on iteration 200
Working on iteration 300
Working on iteration 400
Working on iteration 500
Working on iteration 600
Working on iteration 700
Working on iteration 800
Working on iteration 900
Working on iteration 1000
Working on iteration 1100
Working on iteration 1200
Working on iteration 1300
Working on iteration 1400
Working on iteration 1500
Working on iteration 1600
PCG stopped after: 1613 iterations with relative error 8.3490912222520297e-07.
 38.443679 seconds (195.48 M allocations: 5.300 GB, 27.64% gc time)
Error is: 8.349091212920195e-7
Working on iteration 100
Working on iteration 200
Working on iteration 300
Working on iteration 400
Working on iteration 500
Working on iteration 600
Working on iteration 700
Working on iteration 800
Working on iteration 900
Working on iteration 1000
PCG stopped after: 1051 iterations with relative error 7.44047777512884629433e-07.
 25.911923 seconds (127.69 M allocations: 3.461 GB, 27.82% gc time)
Error is: 7.4

In [23]:
writeToFile("../graphs/se_1/_log_julia.txt", dbg)